In [1]:
# This script is used to pre-process the cropped boarder patches
import os
import numpy as np
import pandas as pd
import rasterio
import sys
import geopandas as gpd
from pathlib import Path
from tqdm import tqdm
from rasterio.mask import mask, raster_geometry_mask
from shapely.geometry import box
from rasterio.enums import Resampling
from itertools import product
from rasterio import windows
import rioxarray
import shapely
import shutil
from osgeo import gdal

path_cur = os.path.abspath('.')
sys.path.append(path_cur)

from os.path import dirname as up

In [2]:
label_file = Path(os.path.join(up(up(path_cur)), 'MarshMapping_local/data/NAIP_data_processing/labels_tmi_modified.csv'))
out_file = Path(os.path.join(up(up(path_cur)), 'MarshMapping_local/data/NAIP_data_processing/labels_tmi_modified_valid.csv'))

In [3]:
df = pd.read_csv(label_file)

In [4]:
testfile = df['patch_name'][0]
rasterio.open(testfile).read().shape

(4, 256, 256)

In [5]:
def filter_image_size(img):

    image = rasterio.open(img).read()
    w,h = image.shape[1:]
    
    if w!=256 or h!=256:
        return 0
    else:
        return 1

def filter_image_zero(img):
    
    image = rasterio.open(img).read()
    
    zero_count = len(np.where(image==0)[0])
    
    if zero_count > 0:
        return 0
    else:
        return 1


In [7]:
df['valid'] = df['patch_name'].apply(lambda x: filter_image_size(x))
df['filter_zero'] = df['patch_name'].apply(lambda x: filter_image_zero(x))

In [11]:
df[df['filter_zero']==1]

,patch_name,label,valid,filter_zero
0,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
1,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
2,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
3,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
4,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
...,...,...,...,...
3870,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
3871,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
3872,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1
3873,/rapids/notebooks/sciclone/geograd/Miranda/git...,/rapids/notebooks/sciclone/geograd/Miranda/git...,1,1


In [12]:
df.to_csv(out_file, index=False)

In [ ]:
# rasterio.open(df['patch_name'][57])
# array_test = rasterio.open(df['patch_name'][2]).read()
# np.where(array_test==0)[0]